In [33]:
# # 删除D:\workspace\xiaoyao\data\stock_minutely_price下的所有文件和子目录
# import os
# import shutil

# # 定义路径
# path = r"D:\workspace\xiaoyao\data\stock_minutely_price"

# # 检查路径是否存在
# if os.path.exists(path):
#     # 删除所有文件和子目录
#     shutil.rmtree(path)
#     print(f"已删除路径: {path}")
# else:
#     print(f"路径不存在: {path}")

# 统计数量

In [1]:
# 统计D:\workspace\xiaoyao\data\stock_minutely_price目录下的子目录数量
import os

dir_path = r'D:\workspace\xiaoyao\data\stock_minutely_price'
sub_dir_count = len(os.listdir(dir_path))
print(sub_dir_count)


5449


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

def generate_minutely_count_csv(root_dir: str, output_csv: str) -> pd.DataFrame:
    """
    读取stock_minutely_price目录，生成minutely_data_count.csv（统计每只股票的分钟数据日期分布）
    """
    print(f"📥 开始读取分钟数据目录：{root_dir}")
    stats_list = []
    
    # 遍历所有股票子目录（格式：stock_code=XXX.XSHE/XSHG）
    for stock_folder in Path(root_dir).iterdir():
        if not stock_folder.is_dir() or "stock_code=" not in stock_folder.name:
            continue
        
        # 提取股票代码
        stock_code = stock_folder.name.split("stock_code=")[-1]
        data_path = stock_folder / "data.parquet"
        
        if not data_path.exists():
            print(f"⚠️ 跳过{stock_code}：缺少data.parquet文件")
            continue
        
        try:
            # 仅读取date列，统计日期分布
            df = pd.read_parquet(data_path, columns=["date"])
            df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
            # 按日期统计数据条数（单股票单日）
            daily_count = df.groupby("date").size().reset_index(name="data_count")
            daily_count["stock_code"] = stock_code
            stats_list.append(daily_count)
            print(f"✅ 处理完成：{stock_code}（{len(daily_count)}天数据）")
        except Exception as e:
            print(f"⚠️ 读取{stock_code}失败：{str(e)[:50]}...")
            continue
    
    # 合并统计结果并保存
    if not stats_list:
        raise ValueError("❌ 未读取到任何有效分钟数据")
    
    minutely_count_df = pd.concat(stats_list, ignore_index=True)
    minutely_count_df = minutely_count_df.sort_values(["stock_code", "date"]).reset_index(drop=True)
    minutely_count_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n💾 minutely_data_count.csv已保存至：{output_csv}")
    print(f"📊 分钟数据统计：共{minutely_count_df['stock_code'].nunique()}只股票，{len(minutely_count_df)}个（日期-股票）组合")
    return minutely_count_df

def compare_stock_code_diff(minutely_count_df: pd.DataFrame, daily_data_path: str) -> None:
    """
    比对分钟数据与日线数据（2025年+paused==0）的股票代码差异
    """
    print(f"\n🔍 开始比对股票代码差异...")
    
    # 1. 提取分钟数据中的所有股票代码（去重）
    minutely_stocks = set(minutely_count_df["stock_code"].unique())
    print(f"分钟数据中的股票数：{len(minutely_stocks)}")
    
    # 2. 提取日线数据中2025年未停牌的股票代码（去重）
    daily_df = pd.read_parquet(daily_data_path)
    daily_df["date"] = pd.to_datetime(daily_df["date"])
    # 筛选条件：2025年 + 未停牌
    daily_2025_paused_0 = daily_df[
        (daily_df["date"].dt.year == 2025) & 
        (daily_df["paused"] == 0)
    ]
    daily_stocks = set(daily_2025_paused_0["stock_code"].unique())
    print(f"日线数据（2025年+未停牌）中的股票数：{len(daily_stocks)}")
    
    # 3. 计算差异
    # 分钟数据有但日线数据无的股票（可能是日线数据缺失或股票已退市）
    min_only_stocks = minutely_stocks - daily_stocks
    # 日线数据有但分钟数据无的股票（需重点关注，可能是分钟数据未抓取）
    daily_only_stocks = daily_stocks - minutely_stocks
    
    # 4. 输出差异结果
    print(f"\n📊 股票代码差异统计：")
    print(f"1. 分钟数据独有股票数：{len(min_only_stocks)}")
    if len(min_only_stocks) > 0:
        print(f"   示例（前10只）：{list(min_only_stocks)[:10]}")
    
    print(f"\n2. 日线数据（2025+未停牌）独有股票数：{len(daily_only_stocks)}")
    if len(daily_only_stocks) > 0:
        print(f"   示例（前10只）：{list(daily_only_stocks)[:10]}")
    
    # 5. 保存差异结果（便于后续处理）
    diff_result = pd.DataFrame({
        "diff_type": ["min_only"] * len(min_only_stocks) + ["daily_only"] * len(daily_only_stocks),
        "stock_code": list(min_only_stocks) + list(daily_only_stocks)
    })
    diff_output = r"D:\workspace\xiaoyao\works\tools\stock_code_diff.csv"
    diff_result.to_csv(diff_output, index=False, encoding="utf-8-sig")
    print(f"\n💾 差异结果已保存至：{diff_output}")

# --------------------------
# 执行入口：生成CSV + 比对差异
# --------------------------
if __name__ == "__main__":
    # 配置路径
    MINUTELY_ROOT_DIR = r"D:\workspace\xiaoyao\data\stock_minutely_price"  # 分钟数据根目录
    MINUTELY_COUNT_CSV = r"D:\workspace\xiaoyao\works\tools\minutely_data_count.csv"  # 生成的统计CSV
    DAILY_DATA_PATH = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet"  # 日线数据路径
    
    try:
        # 1. 生成minutely_data_count.csv
        minutely_count_df = generate_minutely_count_csv(MINUTELY_ROOT_DIR, MINUTELY_COUNT_CSV)
        
        # 2. 比对股票代码差异
        compare_stock_code_diff(minutely_count_df, DAILY_DATA_PATH)
        
        print(f"\n✅ 所有流程完成！")
    except Exception as e:
        print(f"\n❌ 执行失败：{str(e)}")

📥 开始读取分钟数据目录：D:\workspace\xiaoyao\data\stock_minutely_price
✅ 处理完成：000001.XSHE（193天数据）
✅ 处理完成：000002.XSHE（193天数据）
✅ 处理完成：000004.XSHE（193天数据）
✅ 处理完成：000005.XSHE（187天数据）
✅ 处理完成：000006.XSHE（193天数据）
✅ 处理完成：000007.XSHE（193天数据）
✅ 处理完成：000008.XSHE（193天数据）
✅ 处理完成：000009.XSHE（193天数据）
✅ 处理完成：000010.XSHE（193天数据）
✅ 处理完成：000011.XSHE（193天数据）
✅ 处理完成：000012.XSHE（193天数据）
✅ 处理完成：000014.XSHE（193天数据）
✅ 处理完成：000016.XSHE（193天数据）
✅ 处理完成：000017.XSHE（193天数据）
✅ 处理完成：000018.XSHE（187天数据）
✅ 处理完成：000019.XSHE（193天数据）
✅ 处理完成：000020.XSHE（193天数据）
✅ 处理完成：000021.XSHE（193天数据）
✅ 处理完成：000022.XSHE（187天数据）
✅ 处理完成：000023.XSHE（187天数据）
✅ 处理完成：000024.XSHE（187天数据）
✅ 处理完成：000025.XSHE（193天数据）
✅ 处理完成：000026.XSHE（193天数据）
✅ 处理完成：000027.XSHE（193天数据）
✅ 处理完成：000028.XSHE（193天数据）
✅ 处理完成：000029.XSHE（193天数据）
✅ 处理完成：000030.XSHE（193天数据）
✅ 处理完成：000031.XSHE（193天数据）
✅ 处理完成：000032.XSHE（193天数据）
✅ 处理完成：000033.XSHE（187天数据）
✅ 处理完成：000034.XSHE（193天数据）
✅ 处理完成：000035.XSHE（193天数据）
✅ 处理完成：000036.XSHE（193天数据）
✅ 处理完成：000037.XSHE（193天数据）
✅ 处理完成：000038.XSHE（187

KeyboardInterrupt: 

: 

In [1]:
import pandas as pd
from pathlib import Path

def generate_single_date_minutely_count(root_dir: str, target_date: str, output_csv: str) -> pd.DataFrame:
    """
    只统计指定日期的分钟数据分布（如2025-10-23），输出包含股票代码和该日期数据条数的CSV
    
    参数:
        root_dir: 分钟数据根目录（包含stock_code=XXX子文件夹）
        target_date: 目标日期，格式需为"YYYY-MM-DD"
        output_csv: 结果保存路径
    """
    print(f"📅 开始统计目标日期：{target_date} 的分钟数据分布")
    stats_list = []
    
    # 遍历所有股票子目录
    for stock_folder in Path(root_dir).iterdir():
        if not stock_folder.is_dir() or "stock_code=" not in stock_folder.name:
            continue
        
        # 提取股票代码
        stock_code = stock_folder.name.split("stock_code=")[-1]
        data_path = stock_folder / "data.parquet"
        
        if not data_path.exists():
            # print(f"⚠️ 跳过{stock_code}：无data.parquet文件")  # 可选：减少输出加快速度
            continue
        
        try:
            # 仅读取date列，聚焦目标日期
            df = pd.read_parquet(data_path, columns=["date"])
            # 转换日期格式并筛选目标日期
            df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
            target_data = df[df["date"] == target_date]
            
            if not target_data.empty:
                # 统计该日期的分钟数据条数
                data_count = len(target_data)
                stats_list.append({
                    "stock_code": stock_code,
                    "target_date": target_date,
                    "data_count": data_count
                })
                # 每100只股票打印一次进度（减少IO耗时）
                if len(stats_list) % 100 == 0:
                    print(f"🔍 已处理{len(stats_list)}只股票，当前：{stock_code}")
        
        except Exception as e:
            print(f"⚠️ {stock_code}处理失败：{str(e)[:30]}...")
            continue
    
    # 处理结果
    if not stats_list:
        raise ValueError(f"❌ 未找到{target_date}的任何分钟数据")
    
    result_df = pd.DataFrame(stats_list)
    result_df = result_df.sort_values("stock_code").reset_index(drop=True)
    result_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    
    print(f"\n💾 结果已保存至：{output_csv}")
    print(f"📊 统计结果：{target_date}共有{len(result_df)}只股票有分钟数据，平均每只{result_df['data_count'].mean():.1f}条")
    return result_df


# 执行示例
if __name__ == "__main__":
    # 配置参数
    MINUTELY_ROOT_DIR = r"D:\workspace\xiaoyao\data\stock_minutely_price"  # 分钟数据根目录
    TARGET_DATE = "2025-10-28"  # 目标日期
    OUTPUT_CSV = r"D:\workspace\xiaoyao\works\tools\20251023_minutely_count.csv"  # 输出路径
    
    try:
        df = generate_single_date_minutely_count(
            root_dir=MINUTELY_ROOT_DIR,
            target_date=TARGET_DATE,
            output_csv=OUTPUT_CSV
        )
        print("✅ 单个日期统计完成！")
    except Exception as e:
        print(f"❌ 执行失败：{e}")

📅 开始统计目标日期：2025-10-28 的分钟数据分布
🔍 已处理100只股票，当前：000517.XSHE
🔍 已处理200只股票，当前：000670.XSHE
🔍 已处理300只股票，当前：000821.XSHE
🔍 已处理400只股票，当前：000980.XSHE
🔍 已处理500只股票，当前：001373.XSHE
🔍 已处理600只股票，当前：002086.XSHE
🔍 已处理700只股票，当前：002192.XSHE
🔍 已处理800只股票，当前：002297.XSHE
🔍 已处理900只股票，当前：002402.XSHE
🔍 已处理1000只股票，当前：002517.XSHE
🔍 已处理1100只股票，当前：002623.XSHE
🔍 已处理1200只股票，当前：002731.XSHE
🔍 已处理1300只股票，当前：002846.XSHE
🔍 已处理1400只股票，当前：002955.XSHE
🔍 已处理1500只股票，当前：300017.XSHE
🔍 已处理1600只股票，当前：300130.XSHE
🔍 已处理1700只股票，当前：300237.XSHE
🔍 已处理1800只股票，当前：300347.XSHE
🔍 已处理1900只股票，当前：300456.XSHE
🔍 已处理2000只股票，当前：300560.XSHE
🔍 已处理2100只股票，当前：300663.XSHE
🔍 已处理2200只股票，当前：300771.XSHE
🔍 已处理2300只股票，当前：300873.XSHE
🔍 已处理2400只股票，当前：300980.XSHE
🔍 已处理2500只股票，当前：301087.XSHE
🔍 已处理2600只股票，当前：301205.XSHE
🔍 已处理2700只股票，当前：301325.XSHE
🔍 已处理2800只股票，当前：301528.XSHE
🔍 已处理2900只股票，当前：600039.XSHG
🔍 已处理3000只股票，当前：600177.XSHG
🔍 已处理3100只股票，当前：600310.XSHG
🔍 已处理3200只股票，当前：600455.XSHG
🔍 已处理3300只股票，当前：600582.XSHG
🔍 已处理3400只股票，当前：600706.XSHG
🔍 已处理3500只股票，当前：600822.XSHG